**Implementación de algortimo KNN**

`Margot Ines Paco Chipana`

LIBRERIAS

In [26]:
import math
import numpy as np
import pandas as pd

DATOS
1. Instancias
  * Clase 1 (59)
  * Clase 2 (71)
  * Clase 3 (48)
2. Atributos: 13


In [6]:
#from google.colab import drive
#drive.mount('/content/drive')
#ruta = '/content/drive/MyDrive/wine.csv'
data = pd.read_csv('wine.csv')

In [7]:
y=data['class'].values
X = data.drop(columns=['class']).select_dtypes(include=[np.number]).copy()
feature_names = X.columns.tolist()
X = X.values

CUERPO

In [21]:
X_norm = normalizar(X)

#Guardamos el dataset para usarlo en WEKA
data_norm = pd.DataFrame(X_norm, columns=feature_names)
data_norm['class']=y
data_norm.to_csv('wine_norm.csv', index=False)


In [22]:
#Diccionario de las Funciones, útil para la validación cruzada
DIST_FUNCS = {
    "euclidiana": distancia_euclidiana,
    "manhattan": distancia_manhattan,
    "coseno": distancia_coseno,
}

###Experimento

In [27]:
k_list = [1, 3, 5, 7, 9, 11, 15, 17]
metrics = ["euclidiana", "manhattan", "coseno"]
folds = 5
seed = 42

rows = []
best_per_metric = {}

for m in metrics:
    best_acc = -1.0
    best_k = None
    best_cm = None
    for k in k_list:
        acc_cv, cm_cv, labs = cross_validate(X_norm, y, k, m, folds=folds, seed=seed)
        rows.append({"metric": m, "k": k, "cv_accuracy": acc_cv})
        if acc_cv > best_acc:
            best_acc = acc_cv
            best_k = k
            best_cm = cm_cv
    best_per_metric[m] = {"k": best_k, "cv_accuracy": best_acc, "cm_cv": best_cm}

# VALIDACIÓN LOOCV DEL MEJOR k
for m in metrics:
    k_star = best_per_metric[m]["k"]
    acc_loocv, cm_loocv, labs_loocv = loocv(X_norm, y, k_star, m)
    best_per_metric[m]["loocv_accuracy"] = acc_loocv
    best_per_metric[m]["cm_loocv"] = cm_loocv

# RESULTADOS FINALES
print("\n RESULTADOS:")
for m in metrics:
    info = best_per_metric[m]
    print(f"\nMétrica: {m}")
    print(f"  Mejor k = {info['k']}")
    print(f"  Accuracy (CV 5-fold) = {info['cv_accuracy']:.4f}")
    print(f"  Accuracy (LOOCV) = {info['loocv_accuracy']:.4f}")
    print("  Matriz de confusión (CV):\n", info["cm_cv"])
    print("  Matriz de confusión (LOOCV):\n", info["cm_loocv"])


 RESULTADOS:

Métrica: euclidiana
  Mejor k = 15
  Accuracy (CV 5-fold) = 0.9776
  Accuracy (LOOCV) = 0.9775
  Matriz de confusión (CV):
 [[59  0  0]
 [ 2 67  2]
 [ 0  0 48]]
  Matriz de confusión (LOOCV):
 [[59  0  0]
 [ 2 67  2]
 [ 0  0 48]]

Métrica: manhattan
  Mejor k = 17
  Accuracy (CV 5-fold) = 0.9778
  Accuracy (LOOCV) = 0.9775
  Matriz de confusión (CV):
 [[59  0  0]
 [ 2 67  2]
 [ 0  0 48]]
  Matriz de confusión (LOOCV):
 [[59  0  0]
 [ 2 67  2]
 [ 0  0 48]]

Métrica: coseno
  Mejor k = 3
  Accuracy (CV 5-fold) = 0.9722
  Accuracy (LOOCV) = 0.9663
  Matriz de confusión (CV):
 [[59  0  0]
 [ 3 66  2]
 [ 0  0 48]]
  Matriz de confusión (LOOCV):
 [[59  0  0]
 [ 4 65  2]
 [ 0  0 48]]


FUNCIONES

In [9]:
def distancia_euclidiana(vec1, vec2):
    """
    Calcula la distancia euclidiana entre dos vectores (listas de números).
    Esta es la distancia "en línea recta" en un espacio N-dimensional.
    """
    distancia = 0.0
    # Itera sobre cada dimensión (característica)
    for i in range(len(vec1)):
        distancia += (vec1[i] - vec2[i])**2

    # La distancia es la raíz cuadrada de la suma de las diferencias al cuadrado
    return math.sqrt(distancia)

In [10]:
def distancia_manhattan(vec1, vec2):
    """
    Calcula la distancia de Manhattan (o "distancia de la ciudad")
    entre dos vectores. Es la suma de las diferencias absolutas de
    sus componentes.
    """
    distancia = 0.0
    # Itera sobre cada dimensión (característica)
    for i in range(len(vec1)):
        distancia += abs(vec1[i] - vec2[i])

    return distancia

In [11]:
def distancia_coseno(vec1, vec2):
    """
    Calcula la distancia del coseno entre dos vectores.
    Mide el coseno del ángulo entre ellos. Una similitud de 1 (vectores
    idénticos en dirección) da una distancia de 0.
    """
    producto_punto = 0.0
    magnitud_vec1 = 0.0
    magnitud_vec2 = 0.0

    for i in range(len(vec1)):
        producto_punto += vec1[i] * vec2[i]
        magnitud_vec1 += vec1[i]**2
        magnitud_vec2 += vec2[i]**2

    # Tomamos la raíz cuadrada para obtener las magnitudes (normas)
    magnitud_vec1 = math.sqrt(magnitud_vec1)
    magnitud_vec2 = math.sqrt(magnitud_vec2)

    # Evitar división por cero si uno de los vectores es nulo
    if magnitud_vec1 == 0 or magnitud_vec2 == 0:
        return 1.0  # O 0.0 dependiendo de cómo definas la similitud con un vector nulo

    # Calculamos la similitud del coseno
    similitud = producto_punto / (magnitud_vec1 * magnitud_vec2)

    # La distancia es 1 menos la similitud
    # Se ajusta para evitar problemas de precisión (ej. 1.0000001)
    similitud = max(min(similitud, 1.0), -1.0)

    return 1.0 - similitud

In [12]:
def normalizar(X):
  Xmin = X.min(axis=0)
  Xmax = X.max(axis=0)
  denom = (Xmax - Xmin)
  denom[denom == 0] = 1.0  # evita división por cero
  return (X - Xmin) / denom

In [13]:
def knn_one(x, X_train, y_train, k, dist_func):
  #Distancia entre el punto x y todos los del conjunto de entrenamiento
  dists = np.apply_along_axis(lambda r: dist_func(x, r), 1, X_train)
  #indices de los k vencinos más cercanos
  nn_idx = np.argsort(dists)[:k]
  #etiquetas de los k vecinos más cercanos
  nn_labels = y_train[nn_idx]
  #conteo de votos
  conteo = {}
  for label in nn_labels:
    if label not in conteo:
      conteo[label] = 0
    conteo[label] += 1
  #Ordenar las clases según cuántas veces aparecen (mayor a menor)
  top = sorted(conteo.items(), key=lambda item: item[1], reverse=True)
  top_count = top[0][1]
  candidatos = [lab for lab, c in top if c == top_count]
  #desempate: vecino más cercano
  if len(candidatos) == 1:
    return candidatos[0]
  else:
    for idx in nn_idx:
      if y_train[idx] in candidatos:
        return y_train[idx]

In [14]:
def knn_todos(X_test, X_train, y_train, k, dist_func):
  predicciones = []
  for x in X_test:
   clase_predicha = knn_one(x, X_train, y_train, k, dist_func)
   predicciones.append(clase_predicha)
  return predicciones


In [15]:
def accuracy(y_true, y_pred):
  return float(np.mean(y_true == y_pred))

In [16]:
def confusion_matrix(y_true, y_pred, labels=None):
  if labels is None:
    labels = np.unique(np.concatenate((y_true, y_pred)))
  idx = {lab:i for i, lab in enumerate(labels)}
  cm = np.zeros((len(labels), len(labels)), dtype=int)
  for t, p in zip(y_true, y_pred):
    cm[idx[t], idx[p]] += 1
  return cm, labels

###Validación Cruzada (k-fold)

In [17]:
def kfold_indices(n, k, seed=42):
    rng = np.random.default_rng(seed)
    idx = np.arange(n)
    rng.shuffle(idx)
    return np.array_split(idx, k)

In [18]:
def cross_validate(X, y, k_neighbors, dist_name, folds=5, seed=42):
    dist_func = DIST_FUNCS[dist_name]
    fold_idx = kfold_indices(len(X), folds, seed)
    accs, cms = [], []
    labs_ref = None
    for f in range(folds):
        test_idx = fold_idx[f]
        train_idx = np.concatenate([fold_idx[i] for i in range(folds) if i != f])
        #print(f"Fold {f+1}: train={len(train_idx)} muestras, test={len(test_idx)} muestras")
        y_pred = knn_todos(X[test_idx], X[train_idx], y[train_idx], k_neighbors, dist_func)
        accs.append(accuracy(y[test_idx], y_pred))
        cm, labs = confusion_matrix(y[test_idx], y_pred)
        cms.append(cm)
        labs_ref = labs
    avg_acc = float(np.mean(accs))
    sum_cm = np.sum(cms, axis=0)
    return avg_acc, sum_cm, labs_ref

###Validación Leave-one-out (LOOCV)

In [19]:
def loocv(X, y, k_neighbors, dist_name):
    dist_func = DIST_FUNCS[dist_name]
    n = len(X)
    y_pred = np.empty(n, dtype=y.dtype)
    for i in range(n):
        mask = np.ones(n, dtype=bool)
        mask[i] = False
        y_pred[i] = knn_one(X[i], X[mask], y[mask], k_neighbors, dist_func)
    acc = accuracy(y, y_pred)
    cm, labs = confusion_matrix(y, y_pred)
    return acc, cm, labs

#Comparación SKLEARN

In [28]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

# Configuración
mejor_k = 3
metric_sklearn = 'cosine'   # nombre correcto
folds = 5
seed = 42

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
y_pred_all = np.zeros_like(y)

for tr, te in kf.split(X_norm, y):
    X_train, X_test = X_norm[tr], X_norm[te]
    y_train, y_test = y[tr], y[te]

    knn = KNeighborsClassifier(n_neighbors=mejor_k, metric=metric_sklearn)
    knn.fit(X_train, y_train)
    y_pred_all[te] = knn.predict(X_test)

acc_sklearn = accuracy_score(y, y_pred_all)
cm_sklearn = confusion_matrix(y, y_pred_all)

print(f"KNN sklearn (k={mejor_k}, metric={metric_sklearn})")
print(f"Accuracy promedio (5-fold): {acc_sklearn:.4f}")
print("\nMatriz de confusión (acumulada 5-fold):")
print(cm_sklearn)


KNN sklearn (k=3, metric=cosine)
Accuracy promedio (5-fold): 0.9551

Matriz de confusión (acumulada 5-fold):
[[59  0  0]
 [ 6 63  2]
 [ 0  0 48]]
